--- Day 10: Pipe Maze ---

You use the hang glider to ride the hot air from Desert Island all the way up to the floating metal island. This island is surprisingly cold and there definitely aren't any thermals to glide on, so you leave your hang glider behind.

You wander around for a while, but you don't find any people or animals. However, you do occasionally find signposts labeled "Hot Springs" pointing in a seemingly consistent direction; maybe you can find someone at the hot springs and ask them where the desert-machine parts are made.

The landscape here is alien; even the flowers and trees are made of metal. As you stop to admire some metal grass, you notice something metallic scurry away in your peripheral vision and jump into a big pipe! It didn't look like any animal you've ever seen; if you want a better look, you'll need to get ahead of it.

Scanning the area, you discover that the entire field you're standing on is densely packed with pipes; it was hard to tell at first because they're the same metallic silver color as the "ground". You make a quick sketch of all of the surface pipes you can see (your puzzle input).

The pipes are arranged in a two-dimensional grid of tiles:

    | is a vertical pipe connecting north and south.
    - is a horizontal pipe connecting east and west.
    L is a 90-degree bend connecting north and east.
    J is a 90-degree bend connecting north and west.
    7 is a 90-degree bend connecting south and west.
    F is a 90-degree bend connecting south and east.
    . is ground; there is no pipe in this tile.
    S is the starting position of the animal; there is a pipe on this tile, but your sketch doesn't show what shape the pipe has.

Based on the acoustics of the animal's scurrying, you're confident the pipe that contains the animal is one large, continuous loop.

For example, here is a square loop of pipe:

.....
.F-7.
.|.|.
.L-J.
.....

If the animal had entered this loop in the northwest corner, the sketch would instead look like this:

.....
.S-7.
.|.|.
.L-J.
.....

In the above diagram, the S tile is still a 90-degree F bend: you can tell because of how the adjacent pipes connect to it.

Unfortunately, there are also many pipes that aren't connected to the loop! This sketch shows the same loop as above:

-L|F7
7S-7|
L|7||
-L-J|
L|-JF

In the above diagram, you can still figure out which pipes form the main loop: they're the ones connected to S, pipes those pipes connect to, pipes those pipes connect to, and so on. Every pipe in the main loop connects to its two neighbors (including S, which will have exactly two pipes connecting to it, and which is assumed to connect back to those two pipes).

Here is a sketch that contains a slightly more complex main loop:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...

Here's the same example sketch with the extra, non-main-loop pipe tiles also shown:

7-F7-
.FJ|7
SJLL7
|F--J
LJ.LJ

If you want to get out ahead of the animal, you should find the tile in the loop that is farthest from the starting position. Because the animal is in the pipe, it doesn't make sense to measure this by direct distance. Instead, you need to find the tile that would take the longest number of steps along the loop to reach from the starting point - regardless of which way around the loop the animal went.

In the first example with the square loop:

.....
.S-7.
.|.|.
.L-J.
.....

You can count the distance each tile in the loop is from the starting point like this:

.....
.012.
.1.3.
.234.
.....

In this example, the farthest point from the start is 4 steps away.

Here's the more complex loop again:

..F7.
.FJ|.
SJ.L7
|F--J
LJ...

Here are the distances for each tile on that loop:

..45.
.236.
01.78
14567
23...

Find the single giant loop starting at S. How many steps along the loop does it take to get from the starting position to the point farthest from the starting position?


In [6]:
! pip install termcolor

from termcolor import colored


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\dkwar\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [39]:
class Cell:
    def __init__(self, row, col, value):
        self.row = row
        self.col = col
        self.value = value
    
    def __eq__(self, other):
        return self.row == other.row and self.col == other.col
    
    def __str__(self):
        return self.value
    
    def get_possible_movements(self):
        # (row_offset, column_offset)
        offset_mapping = {
            '|': [(-1, 0), (1, 0)],  # Vertical pipe: Move up (-1, 0) or down (1, 0)
            '-': [(0, -1), (0, 1)],  # Horizontal pipe: Move left (0, -1) or right (0, 1)
            'L': [(-1, 0), (0, 1)],  # 90-degree bend: Move up (-1, 0) or right (0, 1)
            'J': [(-1, 0), (0, -1)], # 90-degree bend: Move up (-1, 0) or left (0, -1)
            '7': [(1, 0), (0, -1)],  # 90-degree bend: Move down (1, 0) or left (0, -1)
            'F': [(1, 0), (0, 1)],   # 90-degree bend: Move down (1, 0) or right (0, 1)
            '.': [],                 # Ground: No movement
            'S': []                  # Starting position: The direction is unknown, so no movement (or specific logic could be defined)
        }

        return offset_mapping.get(self.value, []) 

class PipeMaze:
    def __init__(self, input):
        start_char = 'S'
        self.maze = self.__create(input)
        self.start_cell = self.__find_start(start_char, self.maze)
        self.start_neighbours = self.__find_start_neighbours(self.start_cell, self.maze)
        self.path = self.__trace_path(self.start_neighbours, self.maze)
        self.end_cell, self.path_end_steps = self.__find_end(self.path)

    def __create(self, input):
        return [[Cell(row, col, char) for col, char in enumerate(line)] 
            for row, line in enumerate(input.split('\n')) if line]

    def __find_start(self, start_char, maze):
        for row_index, row in enumerate(maze):
            for cell in row:
                if cell.value == start_char:
                    return Cell(row_index, cell.col, start_char)
                
    def __find_start_neighbours(self, start_cell, maze):
        neighbours = []
        row = start_cell.row
        col = start_cell.col
        offsets = [(0, 1), (0, -1), (1, 0), (-1, 0)] # right, left, down, up

        for offset in offsets:
            neighbour_row = row + offset[0]
            neighbour_col = col + offset[1]
            if neighbour_row >= 0 and neighbour_row < len(maze) and neighbour_col >= 0 and neighbour_col < len(maze[0]):
                neighbor = maze[neighbour_row][neighbour_col]
                if neighbor.value != '.':
                    neighbours.append(neighbor)

        return neighbours
    
    def __trace_path(self, neighbours, maze):
        path = []
        previous_cell = self.start_cell
        current_cell = neighbours[0]
        while current_cell != self.start_cell:
            path.append(current_cell)
            current_possible_movements = current_cell.get_possible_movements()
            for movement in current_possible_movements:
                next_cell = maze[current_cell.row + movement[0]][current_cell.col + movement[1]]
                if next_cell != previous_cell:
                    previous_cell = current_cell
                    current_cell = next_cell
                    break
        return path

    def __find_end(self, path):
        steps = (len(path) // 2) + 1
        return path[steps - 1], steps
                
    def __map(self, char):
        mapping = {
            '|': '|',
            '-': '—',
            'L': '└',
            'J': '┘',
            '7': '┐',
            'F': '┌',
            '.': '.',
            'S': 'S',
        }

        return mapping.get(char.value, '?') 
    
    def __str__(self):
        maze_str = ''
        for row in self.maze:
            for cell in row:
                if cell == self.start_cell:
                    maze_str += colored(self.__map(cell), 'green')
                elif cell == self.end_cell:
                    maze_str += colored(self.__map(cell), 'red')
                elif cell in self.path:
                    maze_str += colored(self.__map(cell), 'blue')
                elif cell.value == '.':
                    maze_str += colored(self.__map(cell), 'white')
                else:
                    maze_str += colored(self.__map(cell), 'grey')
                maze_str += ''
            maze_str += '\n'
        return maze_str



input1 = """..F7.
.FJ|.
SJ.L7
|F--J
LJ...
"""

with open('input.txt', 'r') as file:
    input3 = file.read()

maze = PipeMaze(input3)
print("Steps: " + str(maze.path_end_steps))
print(maze)

Steps: 6812
.┐┌┐┐┌—┘—┌.└┌—|—└└.┘—|—┌.┐—|┐┐└—└.┌└┌┌┘┐—┐┐┐┐—|┐—┌┐.└┌┌——┐┌.└┘┐—┘┐┌—┐—┌└—┐┘┐┐|—└—┐—┌—┌——.┘┐..┐┌┌└┘┐..|┐.┌|——┐┌└|—|┐┌——┘———┘┌—└└—└┐—|.|—└┘┐.┌
┌└┐┘—┐┌└└└..|||.└└|┘.┐.└||—┌—┘┘|—┘┐|└┘—┐.└┐└——┐—┐.|┐┌—┘┌┌┘┘┐—|└└└┐┐┘|┌┘|└┐—|—┘┘└—┘┌|||—┘.||┌———┐┐└┐┐┐└┐.┌┘┘└┌—┐┌┐┌└┘..└┌┐—└┐┘└|┌└|—┐┐.┌—|.┌|
┐—┐┌┌.||..┌┘—|┘┘┐|┘┌||—└└|—┘┐┘┘┌┐.|┐|┐└┘┐┐└┐┘|┌—┐┘┘└—┘┐—|┘└┌—┌—┐.└|┘└└┘|—└┐┘┘┘┌┘|┌┐┘—┌┐┘—┌┌┘┘|┌└┐└┐┐—└┘.|┐┘┌—┘┘└┌.└|.—┐┘└┐┐|—┐└└.┐┘┌┐┐—.└—┌.
┐┘└┘┘|┘┌└┌┘.└|└└┌——┘||┐┌┌┌┌—┌.|└|┐┘|┘——|└┘.|—|.┌┘.—|└—┘└.—┘└┌┌.┐┐—┐.└|—┘┐┌┌┐┌┐┌┐┌┘|┐.└└┐┐┌┘|┘.—└┘┐└|—┘└——┐.┘┌|.┌┘—.└—┘┘.└—┌—┐..|└┘—┘┌|┐—┐.└—
└┘┌|.|└———┐┐—┘┐———┐.└┌┐┌┐||.┌——┌.┌—┐..┌┘┐.┘┐—|┌┐┌┘└┘—|└|—|┌┐.└┌—┐—└|└┌┘┐└—|└┘└┘|└┐└┐┘.└┐┐└—┌┌┘.└—┘—┌┌┘.|┌└—┘—┘.|┘└┌┌.┐—┐└└|—└└┌┐|—└┌┘└┌┐└┌└┐
└—└└—┘┌└|.|—┐└┐└┌┌—┌┘|┌┌└|—┌└┐┐└—┘||—┌┐—┌┐┌└—└—┘|┘—└┌—┌|—┐|┌└┘└┐|┌┐|.┌—┐|┌└┐┌—┐└┐|┌┘.┘|.┘┐┘|└┌┐┐—┘┐.|┐┌┘┐┌|..┌└┌┐.|—┌┘┌—┐┌——┌┘||┌.┌┐┐—||—┐┌┘
└┐—└.|┐┌|┌└——┌.┌—┘┘|┐┐|└┘┘|┘┌┘┘┐||┌——┘|┐┐┌┌┐┌└|┐.└..┌┐.|.┌┐└└┐┐|└┘|—┌┘┌┘┐—┌┘|┘└┐||└—┐└┘—└┐┌|┌|.|┌┐└└└┐┐|└┌┐.┌—┘┘└┘.|└┐└┌—┐┘.└.|┐└—|└└┌┐┘┌┘|┘
|

Your puzzle answer was 6812.

The first half of this puzzle is complete! It provides one gold star: *

--- Part Two ---

You quickly reach the farthest point of the loop, but the animal never emerges. Maybe its nest is within the area enclosed by the loop?

To determine whether it's even worth taking the time to search for such a nest, you should calculate how many tiles are contained within the loop. For example:

...........
.S-------7.
.|F-----7|.
.||.....||.
.||.....||.
.|L-7.F-J|.
.|..|.|..|.
.L--J.L--J.
...........

The above loop encloses merely four tiles - the two pairs of . in the southwest and southeast (marked I below). The middle . tiles (marked O below) are not in the loop. Here is the same loop again with those regions marked:

...........
.S-------7.
.|F-----7|.
.||OOOOO||.
.||OOOOO||.
.|L-7OF-J|.
.|II|O|II|.
.L--JOL--J.
.....O.....

In fact, there doesn't even need to be a full tile path to the outside for tiles to count as outside the loop - squeezing between pipes is also allowed! Here, I is still within the loop and O is still outside the loop:

..........
.S------7.
.|F----7|.
.||OOOO||.
.||OOOO||.
.|L-7F-J|.
.|II||II|.
.L--JL--J.
..........

In both of the above examples, 4 tiles are enclosed by the loop.

Here's a larger example:

.F----7F7F7F7F-7....
.|F--7||||||||FJ....
.||.FJ||||||||L7....
FJL7L7LJLJ||LJ.L-7..
L--J.L7...LJS7F-7L7.
....F-J..F7FJ|L7L7L7
....L7.F7||L7|.L7L7|
.....|FJLJ|FJ|F7|.LJ
....FJL-7.||.||||...
....L---J.LJ.LJLJ...

The above sketch has many random bits of ground, some of which are in the loop (I) and some of which are outside it (O):

OF----7F7F7F7F-7OOOO
O|F--7||||||||FJOOOO
O||OFJ||||||||L7OOOO
FJL7L7LJLJ||LJIL-7OO
L--JOL7IIILJS7F-7L7O
OOOOF-JIIF7FJ|L7L7L7
OOOOL7IF7||L7|IL7L7|
OOOOO|FJLJ|FJ|F7|OLJ
OOOOFJL-7O||O||||OOO
OOOOL---JOLJOLJLJOOO

In this larger example, 8 tiles are enclosed by the loop.

Any tile that isn't part of the main loop can count as being enclosed by the loop. Here's another example with many bits of junk pipe lying around that aren't connected to the main loop at all:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJ7F7FJ-
L---JF-JLJ.||-FJLJJ7
|F|F-JF---7F7-L7L|7|
|FFJF7L7F-JF7|JL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L

Here are just the tiles that are enclosed by the loop marked with I:

FF7FSF7F7F7F7F7F---7
L|LJ||||||||||||F--J
FL-7LJLJ||||||LJL-77
F--JF--7||LJLJIF7FJ-
L---JF-JLJIIIIFJLJJ7
|F|F-JF---7IIIL7L|7|
|FFJF7L7F-JF7IIL---7
7-L-JL7||F7|L7F-7F7|
L.L7LFJ|||||FJL7||LJ
L7JLJL-JLJLJL--JLJ.L

In this last example, 10 tiles are enclosed by the loop.

Figure out whether you have time to search for the nest by calculating the area within the loop. How many tiles are enclosed by the loop?
